In [5]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 9.4 MB/s eta 0:00:00m eta 0:00:010:00:01


In [6]:
from __future__ import print_function
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard

# Paths
BASE_DIR = "/home/soggygenus/Documents/CV/devanagari-character-recognition"
TRAIN_DIR = "/home/soggygenus/Documents/CV/devanagari-character-recognition/train/"
TEST_DIR = "/home/soggygenus/Documents/CV/devanagari-character-recognition/test/"
LOG_DIR = "/home/soggygenus/Documents/CV/devanagari-character-recognition/"

# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

training_set = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(32, 32), batch_size=100, class_mode='categorical')
test_set = test_datagen.flow_from_directory(TEST_DIR, target_size=(32, 32), batch_size=100, class_mode='categorical')

# Model
num_classes = 46
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),  # Removed kernel_initializer
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
tensorboard_callback = TensorBoard(log_dir=LOG_DIR, write_graph=True, write_images=True)

# Training
model.fit(training_set, 
          steps_per_epoch=len(training_set), 
          epochs=100, 
          validation_data=test_set, 
          validation_steps=len(test_set), 
          callbacks=[tensorboard_callback])

# Save Model
model.save("devnagri_character_model.keras")


Found 78200 images belonging to 46 classes.
Found 13800 images belonging to 46 classes.


/home/soggygenus/miniconda3/envs/jupyter_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 67s 80ms/step - accuracy: 0.2691 - loss: 2.7140 - val_accuracy: 0.8712 - val_loss: 0.4367
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 56s 71ms/step - accuracy: 0.7486 - loss: 0.8346 - val_accuracy: 0.9336 - val_loss: 0.2290
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 58s 75ms/step - accuracy: 0.8391 - loss: 0.5372 - val_accuracy: 0.9510 - val_loss: 0.1577
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 56s 71ms/step - accuracy: 0.8797 - loss: 0.4160 - val_accuracy: 0.9585 - val_loss: 0.1408
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 65ms/step - accuracy: 0.9007 - loss: 0.3421 - val_accuracy: 0.9678 - val_loss: 0.1076
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 65ms/step - accuracy: 0.9134 - loss: 0.3034 - val_accuracy: 0.9697 - val_loss: 0.1045
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 54s 69ms/step - accuracy: 0.9234 - loss: 0.2656 - val_accuracy: 0.9693 - val_loss: 0.1088
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 54s 69ms/step - accuracy: 0.9311 - loss: 0.2456 - 

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /home/soggygenus/Documents/CV/devanagari-character-recognition


In [ ]:
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os

# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
    '/home/soggygenus/Documents/CV/devanagari-character-recognition/train/',
    target_size=(32, 32),
    batch_size=32
)

# Load model
classifier = load_model('/home/soggygenus/Documents/CV/devanagari-character-recognition/devnagri_character_model.keras')
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Class Indices
classes = training_set.class_indices

# Load Image
image_path = input("Enter image name & path (without .png extension): ") + ".png"
if not os.path.exists(image_path):
    print("Error: File not found!")
    exit()

testImage = cv2.imread(image_path)
testImage = cv2.resize(testImage, (32, 32))
testimage = np.array(testImage) / 255.0  # Normalize
testimage = np.expand_dims(testimage, axis=0)  # Add batch dimension

# Display Image
plt.title("Input Image")
plt.imshow(cv2.cvtColor(testImage, cv2.COLOR_BGR2RGB))
plt.show()

# Predict Class
prediction = np.argmax(classifier.predict(testimage), axis=1)

# Output Result
for key, value in classes.items():
    if value == prediction:
        print("Predicted Class:", key)
